<a href="https://colab.research.google.com/github/hakanskn/Fine-Tuned-LLM/blob/main/Vize2_revised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Info

Aşağıda, çalışmada kullandığım kod altyapısının linki yer almaktadır.

In [ ]:
# https://github.com/aybaras/BERT-base-Turkish-QA/blob/main/train.ipynb

# Libraries & Configuration

Gerekli kütüphaneler yükleniyor...

In [ ]:
%%capture

!pip install transformers
!pip install torch

!pip install datasets

# FOR XLM-R
!pip install sentencepiece

Transformers github reposu, lokale kopyalanıyor...

Bu repoda yer alan squad.py dosyası ile eğitim gerçekleştirilecektir.

In [ ]:
!git clone https://github.com/huggingface/transformers
!cd transformers

Cloning into 'transformers'...
remote: Enumerating objects: 202797, done.
remote: Counting objects: 100% (263/263), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 202797 (delta 159), reused 239 (delta 147), pack-reused 202534
Receiving objects: 100% (202797/202797), 213.49 MiB | 13.50 MiB/s, done.
Resolving deltas: 100% (145285/145285), done.


Lokalde verisetinin kopyalanacağı dizin hazırlanıyor...

In [ ]:
%env DATA_DIR=/content/Squad1

env: DATA_DIR=/content/Squad1


# Data Loading & Preprocessing

Bu çalışmada aşağıdaki veri seti kullanılmıştır.

[TQuAD (Turkish Question Answering Data Set) ](https://github.com/TQuad/turkish-nlp-qa-dataset)


Aşağıdaki veri seti indirilmiş, yukarıdaki veri seti ile birleştirilmiş ancak vakit yetmediği için kullanılamamıştır.

[Turkish Reading Comprehension Question Answering Data Set](https://github.com/okanvk/Turkish-Reading-Comprehension-Question-Answering-Dataset)


kuzgunlar veri seti çekilmiş ancak işleme alınmamıştır.

[Kuzgunlar's Data Set](https://github.com/kuzgnlar/datasets)

In [ ]:
%%capture
!wget -P $DATA_DIR https://raw.githubusercontent.com/TQuad/turkish-nlp-qa-dataset/master/dev-v0.1.json
!wget -P $DATA_DIR https://raw.githubusercontent.com/TQuad/turkish-nlp-qa-dataset/master/train-v0.1.json
!wget -P $DATA_DIR https://raw.githubusercontent.com/kuzgnlar/datasets/master/question-answer/kuzgunlar_data.json

!wget -P $DATA_DIR https://raw.githubusercontent.com/okanvk/Turkish-Reading-Comprehension-Question-Answering-Dataset/master/data/2018%20%2B%202020%20veri%20k%C3%BCmesi/final_dev_data_v2.json
!wget -P $DATA_DIR https://raw.githubusercontent.com/okanvk/Turkish-Reading-Comprehension-Question-Answering-Dataset/master/data/2018%20%2B%202020%20veri%20k%C3%BCmesi/final_train_data_v2.json

Veri setleri lokale indirilip birleştirme işlemi yapılıyor...

In [ ]:
import json

with open("/content/Squad1/train-v0.1.json") as train_tr:
  tr_train_v01 = json.load(train_tr)
with open("/content/Squad1/final_train_data_v2.json") as train_tr:
  tr_train_v02 = json.load(train_tr)
with open("/content/Squad1/dev-v0.1.json") as dev_tr:
  tr_dev_v01 = json.load(dev_tr)
with open("/content/Squad1/final_dev_data_v2.json") as dev_tr:
  tr_dev_v02 = json.load(dev_tr)

In [ ]:
tr_train_data_v01 = tr_train_v01["data"]
tr_train_data_v02 = tr_train_v02["data"]
tr_dev_data_v01 = tr_dev_v01["data"]
tr_dev_data_v02 = tr_dev_v02["data"]

In [ ]:
mixed_train_data = tr_train_data_v01 + tr_train_data_v02
mixed_dev_data = tr_dev_data_v01 + tr_dev_data_v02

In [ ]:
mixed_train = {"version":"v0.1"}
mixed_train["data"] = mixed_train_data
mixed_dev = {"version":"v0.1"}
mixed_dev["data"] = mixed_dev_data

In [ ]:
mixed_dev.keys()

dict_keys(['version', 'data'])

In [ ]:
with open("/content/Squad1/mixed_train_dataset.json","w") as mix_t:
  json.dump(mixed_train,mix_t)
with open("/content/Squad1/mixed_dev_dataset.json","w") as mix_d:
  json.dump(mixed_dev,mix_d)

## Showing samples of Dataset

Bu çalışmada soru-cevap şeklinde hazırlanmış, yukarıda detayları yer alan veri setleri değerlendirilmiştir. Aşağıda veri setinden örnekler gösterilmektedir.

Bu veri seti, "İslamda bilim ve teknik" adlı kitaptan elde edilmiştir. ilgili kaynağın ilk cildine [buradan](https://www.tuba.gov.tr/files/yayinlar/T%C4%B0BKM/%C4%B0slamda%20Bilim%20ve%20Teknik.pdf) ulaşılabilir.



In [37]:
mixed_train['data'][0]['title']

'İslamda bilim ve teknik'

In [47]:
mixed_train['data'][0]['paragraphs'][0]['context']

'İslam dünyasında bilimin 16. yüzyılda hala yüksek seviyede bulunduğunu gösteren çok ilginç bir örneği deskriptif coğrafya ekolünden verebiliriz. Bize bu örneği, Avrupa’da Afrikalı Leo (Leo Africanus) olarak tanınan el-hasan b. Muhammed el-Vezzan (doğumu yaklaşık 888/1483)’dır. Fas (Fez) şehrinde büyümüş ve eğitimini almış olan Granada doğumlu bu bilgin, diplomatik hizmetler yoluyla, özellikle kuzey Afrika’da olmak üzere birçok İslam ülkesini tanıyıp bir yazar olarak coğrafya ve kartografya ile ilgileniyordu. İstanbul’dan dönüş yolculuğunda Sicilyalı korsanların eline esir düşmüş, ilk olarak Napoli’ye daha sonra Roma’ya satılıp Papa X. Leo tarafından 6.1.1520 yılında bizzat Papa’nın adıyla Giovanni Leo olarak vaftiz edilmişti. İtalya’daki ikameti sırasında İtalyanca öğrendi ve Arapça öğretti. Yazar olarak faaliyetlerini Roma ve Bologna’da devam ettirdi. Afrika coğrafyası dışında kuzey Afrikalı 30 bilginin biyografilerini içeren diğer bir eser derledi. Afrika kitabını esaretinin 6. yılı

In [48]:
mixed_train['data'][0]['paragraphs'][1]

{'qas': [{'question': 'el-Hasan b. Muhammed el-Vezzan’ın Afrika kitabı kaç bölümden oluşmaktadır ?',
   'id': 10,
   'answers': [{'answer_start': '17', 'text': '9'}]},
  {'question': 'el-Hasan b. Muhammed el-Vezzan’ın Afrika kitabının birinci bölümünde ne anlatılmaktadır ?',
   'id': 11,
   'answers': [{'answer_start': '56',
     'text': 'Afrika’nın ve Afrika halkının genel fiziksel ve iklimsel özelliklerini'}]},
  {'question': 'el-Hasan b. Muhammed el-Vezzan’ın Afrika kitabının ikinci bölümünde ne anlatılmaktadır ?',
   'id': 12,
   'answers': [{'answer_start': '154',
     'text': 'Marrakuş (Marrakesch) bölgesini şehirleriyle ve dağlarıyla ele almaktadır.'}]},
  {'question': 'el-Hasan b. Muhammed el-Vezzan’ın Afrika kitabının üçüncğ bölümünde ne anlatılmaktadır ?',
   'id': 13,
   'answers': [{'answer_start': '242', 'text': 'Fas’ı'}]},
  {'question': 'el-Hasan b. Muhammed el-Vezzan’ın Afrika kitabının dördüncü bölümünde ne anlatılmaktadır ?',
   'id': 14,
   'answers': [{'answer_start

In [50]:
sample_context = mixed_train['data'][0]['paragraphs'][1]['context']
sample_context

'Söz konusu kitap 9 bölümden oluşmaktadır. Birinci bölüm Afrika’nın ve Afrika halkının genel fiziksel ve iklimsel özelliklerini işlemektedir. İkinci bölüm Marrakuş (Marrakesch) bölgesini şehirleriyle ve dağlarıyla ele almaktadır. Üçüncü bölüm Fas’ı, dördüncü bölüm Tilimsan’ı (Tlemcen), beşinci bölüm Tunus’u, altıncı bölüm Libya’yı, yedinci bölüm Sudan’ı sekizinci bölüm Mısır’ı ve dokuzuncu bölüm Afrika’nın ırmaklarını, yeraltı zenginliklerini, bitki örtüsünü ve hayvanlarını ele almaktadır. Bu kitapta toplam 400 yer tanıtılmaktadır. Yazar, çoğunlukla kendi gözlemlerine dayandığını ve bizzat bildiremeyeceği durumlarda ise en doğru ve kesin bilgileri güvenilir kişilerden almaya çaba sarfettiğini not etmektedir. Leo Africanus’un Afrika tasviri el-İdrisi’nin Nuzhet el-Müştak’ı yanında, 16. yüzyılın ikinci yarısından itibaren Avrupa’da Afrika’nın beşeri coğrafyasının geliştirilmesi ve genişletilmesinde kullanılan en önemli kaynaklardandır. G.B. Ramusio tarafından 1550 yılındaki basımından ço

veri seti soru-cevap şeklinde çiftlerden oluşmaktadır.

örnek:

soru: el-Hasan b. Muhammed el-Vezzan’ın Afrika kitabı kaç bölümden oluşmaktadır ?

cevap: 9 bölümde

In [54]:
mixed_train['data'][0]['paragraphs'][1]['qas'][0]['question']

'el-Hasan b. Muhammed el-Vezzan’ın Afrika kitabı kaç bölümden oluşmaktadır ?'

In [56]:
mixed_train['data'][0]['paragraphs'][1]['qas'][0]['answers']

[{'answer_start': '17', 'text': '9'}]

In [61]:
answer_start = mixed_train['data'][0]['paragraphs'][1]['qas'][0]['answers'][0]['answer_start']
text_count = mixed_train['data'][0]['paragraphs'][1]['qas'][0]['answers'][0]['text']

In [64]:
sample_context[int(answer_start):int(answer_start) + int(text_count)]

'9 bölümde'

# Training

aşağıdaki hücrede

In [ ]:
cmd = [
       'python',
#    '-m torch.distributed.launch --nproc_per_node 2', # use this to perform distributed training over multiple GPUs
    'run_squad.py',

    '--model_type', 'bert',                            # model type (one of the list under "Pick a Model" above)

    '--model_name_or_path', 'bert-base-tr-qa-v1',       # specific model name of the given model type (shown, a list is here: https://huggingface.co/transformers/pretrained_models.html)
                                                       # on first execution this initiates a download of pre-trained model weights;
                                                       # can also be a local path to a directory with model weights

    '--output_dir', './models/bert/aybars_checkpoints',        # directory for model checkpoints and predictions

#    '--overwrite_output_dir',                         # use when adding output to a directory that is non-empty --
                                                       # for instance, when training crashes midway through and you need to restart it

    '--do_train',                                      # execute the training method

    '--train_file', '$DATA_DIR/train-v0.1.json',       # provide the training data

    # '--version_2_with_negative',                       # ** MUST use this flag if training on SQuAD 2.0! DO NOT use if training on SQuAD 1.1

    # '--do_lower_case',                                 # ** set this flag if using an uncased model; don't use for Cased Models

    # '--do_eval',                                       # execute the evaluation method on the dev set -- note:
                                                       # if coupled with --do_train, evaluation runs after fine-tuning

    '--predict_file', '$DATA_DIR/dev-v0.1.json',       # provide evaluation data (dev set)

    '--eval_all_checkpoints',                          # evaluate the model on the dev set at each checkpoint

    '--per_gpu_eval_batch_size', '12',                 # evaluation batch size for each gpu

    '--per_gpu_train_batch_size', '12',                # training batch size for each gpu

    '--save_steps', '5000',                            # how often checkpoints (complete model snapshot) are saved

    '--threads', '8',                                  # num of CPU threads to use for converting SQuAD examples to model features

    # --- Model and Feature Hyperparameters ---
    '--num_train_epochs', '2',                         # number of training epochs - usually 2-3 for SQuAD

    '--learning_rate', '3e-5',                         # learning rate for the default optimizer (Adam in this case)

    '--max_seq_length', '512',                         # maximum length allowed for the full input sequence

    '--doc_stride', '128'                              # used for long documents that must be chunked into multiple features --
       ]

## Data Preparing

In [ ]:
import json

with open("/content/Squad1/dev-v0.1.json") as dev:
  dev_tquad = json.load(dev)

with open("/content/Squad1/train-v0.1.json") as train:
  train_tquad = json.load(train)

def answer_start_to_int(dataset):
  for i in range(len(dataset["data"])):
    for j in range(len(dataset["data"][i]["paragraphs"])):
      for k in range(len(dataset["data"][i]["paragraphs"][j]["qas"])):
        for l in range(len(dataset["data"][i]["paragraphs"][j]["qas"][k]["answers"])):
          dataset["data"][i]["paragraphs"][j]["qas"][k]["answers"][l]["answer_start"] = int(dataset["data"][i]["paragraphs"][j]["qas"][k]["answers"][l]["answer_start"])

answer_start_to_int(train_tquad)
answer_start_to_int(dev_tquad)

train_tquad["version"] = "v0.1"
dev_tquad["version"] = "v0.1"
with open("/content/Squad1/train-v0.1-wv.json","w") as train:
  json.dump(train_tquad,train)

with open("/content/Squad1/dev-v0.1-wv.json","w") as dev:
  json.dump(dev_tquad,dev)

## Train Process

Aşağıda eğitim için örnek bir kod parçası var. Her bir parametre açıklaması yer almaktadır. Ancak eğitimde bir anlamı yok. sadece bilgilendirme için.

In [ ]:
cmd = [
       'python',
#    '-m torch.distributed.launch --nproc_per_node 2', # use this to perform distributed training over multiple GPUs
    'run_squad.py',

    '--model_type', 'bert',                            # model type (one of the list under "Pick a Model" above)

    '--model_name_or_path', 'bert-base-tr-qa-v1',       # specific model name of the given model type (shown, a list is here: https://huggingface.co/transformers/pretrained_models.html)
                                                       # on first execution this initiates a download of pre-trained model weights;
                                                       # can also be a local path to a directory with model weights

    '--output_dir', './models/bert/aybars_checkpoints',        # directory for model checkpoints and predictions

#    '--overwrite_output_dir',                         # use when adding output to a directory that is non-empty --
                                                       # for instance, when training crashes midway through and you need to restart it

    '--do_train',                                      # execute the training method

    '--train_file', '$DATA_DIR/train-v0.1.json',       # provide the training data

    # '--version_2_with_negative',                       # ** MUST use this flag if training on SQuAD 2.0! DO NOT use if training on SQuAD 1.1

    # '--do_lower_case',                                 # ** set this flag if using an uncased model; don't use for Cased Models

    # '--do_eval',                                       # execute the evaluation method on the dev set -- note:
                                                       # if coupled with --do_train, evaluation runs after fine-tuning

    '--predict_file', '$DATA_DIR/dev-v0.1.json',       # provide evaluation data (dev set)

    '--eval_all_checkpoints',                          # evaluate the model on the dev set at each checkpoint

    '--per_gpu_eval_batch_size', '12',                 # evaluation batch size for each gpu

    '--per_gpu_train_batch_size', '12',                # training batch size for each gpu

    '--save_steps', '5000',                            # how often checkpoints (complete model snapshot) are saved

    '--threads', '8',                                  # num of CPU threads to use for converting SQuAD examples to model features

    # --- Model and Feature Hyperparameters ---
    '--num_train_epochs', '2',                         # number of training epochs - usually 2-3 for SQuAD

    '--learning_rate', '3e-5',                         # learning rate for the default optimizer (Adam in this case)

    '--max_seq_length', '512',                         # maximum length allowed for the full input sequence

    '--doc_stride', '128'                              # used for long documents that must be chunked into multiple features --
       ]

Eğitim için xlm-roberta modeli kullanılmıştır. Bunun yerine bert de kullanılabilirdi. Ancak bunun için vakit yoktu. Vize sonrası tüm kombinasyonlar için çalıştırma yapılabilir. (kombinasyon: hem veri setini değiştirme hem de modeli değiştirme)

Aşağıdaki script bize eğitim için gerekli konfigurasyonu veriyor.

In [ ]:
!python /content/transformers/examples/legacy/question-answering/run_squad.py  \
    --model_type xlm-roberta   \
    --model_name_or_path xlm-roberta-base  \
    --output_dir /models/bert/ \
    --data_dir /content/Squad1   \
    --overwrite_output_dir \
    --overwrite_cache \
    --do_train  \
    --train_file  /content/Squad1/train-v0.1-wv.json   \
    --do_eval   \
    --predict_file /content/Squad1/dev-v0.1-wv.json  \
    --per_gpu_train_batch_size 8 \
    --learning_rate 3e-5   \
    --num_train_epochs 2.0   \
    --max_seq_length 512   \
    --doc_stride 128   \
    --threads 10   \
    --save_steps 5000

2024-05-01 14:36:37.639628: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-01 14:36:37.639686: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-01 14:36:37.641793: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-01 14:36:38.742528: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
05/01/2024 14:36:39 - WARNING - __main__ - Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
config.json: 100% 615/615 [00:00<00:00, 3.51MB/s]
[INFO|configuration_utils.py:726] 2024-05-01 14:36:40

Aşağıdaki kodda "--train_file  /content/Squad1/mixed_train_dataset.json" satırına bakıldığında tüm veri setlerinin birleştirildiği dataset üzerinde eğitim yapılacağı anlaşılmaktadır.

In [ ]:
# !python /content/transformers/examples/legacy/question-answering/run_squad.py  \
#     --model_type xlm-roberta   \
#     --model_name_or_path xlm-roberta-base  \
#     --output_dir /models/bert/ \
#     --data_dir /content/Squad1   \
#     --overwrite_output_dir \
#     --overwrite_cache \
#     --do_train  \
#     --train_file  /content/Squad1/mixed_train_dataset.json   \
#     --do_eval   \
#     --predict_file /content/Squad1/mixed_dev_dataset.json  \
#     --per_gpu_train_batch_size 8 \
#     --learning_rate 3e-5   \
#     --num_train_epochs 2.0   \
#     --max_seq_length 512   \
#     --doc_stride 128   \
#     --threads 10   \
#     --save_steps 5000

# Loading Fine-Tuned Model & Test

In [15]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

# Load the fine-tuned model
tokenizer = AutoTokenizer.from_pretrained("/models/bert/")
model = AutoModelForQuestionAnswering.from_pretrained("/models/bert/")

In [72]:
test_question = "Mehmet Akif hangi ilden milletvekili seçilmiştir?"
test_text = """

Malum siyasete uzak bir kişidir Akif. O da bunu kabul etti. Akif ’i bildiğim
kadarıyla İzmit, Biga (Çanakkale’nin bugün bir ilçesi ama o zaman ilçe değildi,
sancaktı) ve Burdur’dan seçildi. Akif Burdur milletvekilliğini tercih etti. Ve
o zamanki usul şuydu yani bugün seçimden sonra ne yapıyoruz? Mazbatayı
alıyoruz. Büyük Millet Meclisinin sekretaryasına müracaat ediyoruz. Bizim
mazbatamızı kabul ediyorlar ve milletvekili oluyoruz. Hâlbuki o zaman Büyük
Millet Meclisi oylama yapıyor. Sizi milletvekili kabul edip etmeme yetkisine
sahip. Mehmet Akif oy birliği ile 6 Haziran 1920’de Büyük Millet Meclisine
Burdur milletvekilli olarak kabul edildi. Ve o Meclis’in kayıt defterine diğer
milletvekilleri mebuslar gibi işte unvanı, şöhreti, mesleği ile yazılıyor milletvekilleri;
Mehmet Akif ’in unvanı olarak da “İslam Şairi” yazıyorlar, “İslam Şairi
Akif Bey”. Evet, Akif Bey Mecliste “İslam Şairi” olarak bulundu.

"""

In [73]:
from transformers import pipeline
question_answerer3 = pipeline("question-answering",model = model, tokenizer = tokenizer)
question_answerer3(question = test_question, context = test_text)

{'score': 0.9777342081069946, 'start': 172, 'end': 178, 'answer': 'Burdur'}

In [74]:
from transformers import pipeline
question_answerer2 = pipeline("question-answering",model = 'savasy/bert-base-turkish-squad')
question_answerer2(question = test_question, context = test_text)

Some weights of the model checkpoint at savasy/bert-base-turkish-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'score': 0.962083637714386, 'start': 172, 'end': 178, 'answer': 'Burdur'}

Yukarıdaki iki sonucu karşılaştırdığımızda fine-tuned edilmiş model, diğer modele göre daha iyi bir skor üretmiştir (0.9777 > 0.9620)

## Another Test

In [75]:
test_question = "Mehmet Akif ’in unvanı nedir?"
test_text = """

Malum siyasete uzak bir kişidir Akif. O da bunu kabul etti. Akif ’i bildiğim
kadarıyla İzmit, Biga (Çanakkale’nin bugün bir ilçesi ama o zaman ilçe değildi,
sancaktı) ve Burdur’dan seçildi. Akif Burdur milletvekilliğini tercih etti. Ve
o zamanki usul şuydu yani bugün seçimden sonra ne yapıyoruz? Mazbatayı
alıyoruz. Büyük Millet Meclisinin sekretaryasına müracaat ediyoruz. Bizim
mazbatamızı kabul ediyorlar ve milletvekili oluyoruz. Hâlbuki o zaman Büyük
Millet Meclisi oylama yapıyor. Sizi milletvekili kabul edip etmeme yetkisine
sahip. Mehmet Akif oy birliği ile 6 Haziran 1920’de Büyük Millet Meclisine
Burdur milletvekilli olarak kabul edildi. Ve o Meclis’in kayıt defterine diğer
milletvekilleri mebuslar gibi işte unvanı, şöhreti, mesleği ile yazılıyor milletvekilleri;
Mehmet Akif ’in unvanı olarak da “İslam Şairi” yazıyorlar, “İslam Şairi
Akif Bey”. Evet, Akif Bey Mecliste “İslam Şairi” olarak bulundu.

"""
question_answerer3(question = test_question, context = test_text)

{'score': 0.39569225907325745,
 'start': 814,
 'end': 826,
 'answer': '“İslam Şairi'}

# Summary

- Daha önce veri setleri taranmış ve probleme uygun veri setleri listelenmiştir. Detayları yukarıda yer almaktadır. Çalışmada TQuad seçilmiştir.
- Probleme özgü modeller taranmış, XLM Roberta ve BERT adlı iki model seçilmiştir. Bu çalışma XLM Roberta üzerinde yapılmıştır.
- Eğitim sonucunda oluşan model ile testler yapılmıştır. Buna uygun olarak bir context (içerik) bir de soru verilmiş, modelden soruya cevap vermesi beklenmiştir.
- Geliştirilen bir başka model ile fine-tuned edilen modelin cevapları aynı context-soru üzerinde karşılaştırılmıştır.



# Saving Model

In [28]:
!zip -r /content/bert_folder.zip  /models



  adding: models/ (stored 0%)
  adding: models/bert/ (stored 0%)
  adding: models/bert/sentencepiece.bpe.model (deflated 49%)
  adding: models/bert/tokenizer_config.json (deflated 76%)
  adding: models/bert/training_args.bin (deflated 49%)
  adding: models/bert/config.json (deflated 50%)
  adding: models/bert/special_tokens_map.json (deflated 52%)
  adding: models/bert/predictions_.json (deflated 68%)
  adding: models/bert/model.safetensors (deflated 30%)
  adding: models/bert/nbest_predictions_.json (deflated 87%)


In [ ]:
from google.colab import files
files.download("/content/bert_folder.zip")

# Other Train Script with Different Datasets and Models

In [ ]:

# whole
# !python /content/transformers/examples/legacy/question-answering/run_squad.py  \
#     --model_type  bert  \
#     --model_name_or_path Aybars/ModelOnTquad \
#     --output_dir /models_eval/bert/ \
#     --overwrite_output_dir \
#     --do_eval   \
#     --overwrite_cache \
#     --predict_file /content/Squad1/whole_dev_dataset.json \
#     --per_gpu_eval_batch_size 8 \
#     --learning_rate 3e-5   \
#     --max_seq_length 512   \
#     --doc_stride 128   \
#     --threads 10   \
#     --save_steps 5000

In [ ]:
# Okanvk
# !python /content/transformers/examples/legacy/question-answering/run_squad.py  \
#     --model_type bert   \
#     --model_name_or_path Aybars/ModelOnTquad \
#     --output_dir /models_eval2/bert/ \
#     --overwrite_output_dir \
#     --overwrite_cache \
#     --do_eval   \
#     --predict_file /content/Squad1/final_dev_data_v2.json \
#     --per_gpu_eval_batch_size 8 \
#     --learning_rate 3e-5   \
#     --max_seq_length 512   \
#     --doc_stride 128   \
#     --threads 10   \
#     --save_steps 5000

In [ ]:
## Evaluating Our model in different dev_datas
# TQUAD
# !python /content/transformers/examples/legacy/question-answering/run_squad.py  \
#     --model_type bert  \
#     --model_name_or_path Aybars/ModelOnTquad \
#     --output_dir /models_eval/bert/ \
#     --overwrite_output_dir \
#     --do_eval   \
#     --overwrite_cache \
#     --predict_file /content/Squad1/dev-v0.1-wv.json \
#     --per_gpu_eval_batch_size 8 \
#     --learning_rate 3e-5   \
#     --max_seq_length 512   \
#     --doc_stride 128   \
#     --threads 10   \
#     --save_steps 5000

In [ ]:
# Xquad.tr
# !python /content/transformers/examples/legacy/question-answering/run_squad.py  \
#     --model_type bert   \
#     --model_name_or_path  Aybars/ModelOnTquad \
#     --output_dir /models_eval3/bert/ \
#     --overwrite_output_dir \
#     --overwrite_cache \
#     --do_eval   \
#     --predict_file /content/Squad1/xquad.tr.json \
#     --per_gpu_eval_batch_size 8 \
#     --learning_rate 3e-5   \
#     --max_seq_length 512   \
#     --doc_stride 128   \
#     --threads 10   \
#     --save_steps 5000